In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import dxchange
import functools

In [24]:
def makeGaussian(size, fwhm = 3, center=None):
    x = np.arange(0, size, 1, float)
    y = x[:,np.newaxis]
    if center is None:
        x0 = y0 = size // 2
    else:
        x0 = center[0]
        y0 = center[1]
    return np.exp(-4*np.log(2) * ((x-x0)**2 + (y-y0)**2) / fwhm**2)
def calc_diffractionpattern(obj, probe, shift, N_ph=1.e7):
    ny, nx  = np.shape(probe)
    probe *= np.sqrt(1./np.sum(np.abs(probe)**2)*N_ph/ny/nx)
    obj_cut = obj[shift[1]:shift[1]+ny, shift[0]:shift[0]+nx]
    Psi = np.fft.ifftshift(np.fft.fft2(np.fft.fftshift(obj_cut*probe)))
    I = np.abs(Psi)**2
    return np.random.poisson(I)
def obj_patch(obj, probe, shift):
    ny, nx  = np.shape(probe)
    obj_p = obj[shift[1]:shift[1]+ny, shift[0]:shift[0]+nx]
    return obj_p

def ptycho_forward(obj, probe, step_size, probe_size):
#     h_probe, w_probe = probe.shape
   
    patches = tf.image.extract_patches(obj, 
                                       sizes = [1, probe_size, probe_size, 1], 
                                       strides = [1, 10, 10, 1],
                                       rates=[1, 1, 1, 1],
                                       padding='SAME')
    print(patches)

    obj_probe = tf.multiply(patches, probe)
    psi = tf.signal.fft3d(patches)
    psi = tf.roll(psi, [probe_size//2, probe_size//2], [1, 2])
    return tf.square(tf.abs(psi))
    
    

def phase_fraunhofer(phase, absorption):
    wf = tf.complex(absorption, phase)
    # wf = tf.complex(phase, absorption)

    # wf = mask_img(wf)
    # wf = tf.multiply(ampl, tf.exp(phshift))
    # wf = tf.manip.roll(wf, [160, 160], [0, 1])
    ifp = tf.square(tf.abs(tf.signal.fft2d(wf)))
    ifp = tf.roll(ifp, [256, 256], [0, 1])
    ifp = tf.reshape(ifp, [ifp.shape[0], ifp.shape[1], 1])
    ifp = tf.image.per_image_standardization(ifp)
    ifp = tfnor_phase(ifp)
    return ifp

In [10]:
def siemensstar(N, R0=80, delta=0.05, segments=18, rings_N=10, rings_f=2., downsample=1):  
    if downsample > 1:
        N = int(N*downsample)
        R = R0*downsample
    else:
        R = R0
    y,x = np.mgrid[:N,:N] - N/2
    phi = np.arctan2(-y,x)
    r = np.sqrt(x**2+y**2)
    stripes = 1.0*(np.sin(segments*phi)**2>0.5)

    center = 1.0*(r<R)
    outer = 1.0*(r>R*(rings_f**rings_N))
    rings = np.zeros_like(stripes)

    if rings_N > 1:
        for i in range(rings_N):
            R *= rings_f
            rings += 1.0*(r>=(1-delta)*R)*(r<R)
    star = functools.reduce(np.maximum, [center, rings, stripes, outer])
    if downsample==1:
        return star
    else:
        return np.average(np.average(star.reshape(N/downsample, downsample, N/downsample, downsample), axis=3), axis=1)

In [26]:
N_object = 256
N_probe = 64
step_size = 20
N_ph=1.e7
data = dxchange.read_tiff('/data/gan_phase/test.tiff')
data = data[128:-128, 128:-128]
print(data.shape, data.max(), data.min())
obj_phase = data

obj_amp = siemensstar(N_object, R0=80, delta=0.05, segments=18, rings_N=10, rings_f=2., downsample=1)


obj = obj_amp*tf.cos(-obj_phase) + 1j*obj_amp*tf.sin(-obj_phase)
obj = tf.reshape(obj, [1, 256, 256, 1])
pro_amp = makeGaussian(N_probe, fwhm=(N_probe*0.25))
# obj = obj_amp*np.cos(-obj_phase) + 1j*obj_amp*np.sin(-obj_phase)
probe = pro_amp*np.cos(0.05*pro_amp) + 1j*pro_amp*np.sin(0.05*pro_amp)
ny, nx  = np.shape(probe)
probe *= np.sqrt(1./np.sum(np.abs(probe)**2)*N_ph/ny/nx)
probe = tf.reshape(probe, [1, 64, 64, 1])
i_diffraction = ptycho_forward(obj, probe, step_size, 64)


print('done calculating shifts')
print(positions.dtype)

(256, 256) 0.0 -1.1220579147338867
tf.Tensor(
[[[[0.         0.         0.         ... 0.50871913 0.50827149
    0.50878738]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.50464836 0.50335269
    0.50389719]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.
    0.       

InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a double tensor but is a complex128 tensor [Op:Mul]